In [1]:
# Importing packages and loading in the data set 
from utils_pos import get_word_tag, preprocess  
import pandas as pd
from collections import defaultdict
import math
import numpy as np

In [2]:
# load in the training corpus
with open("round_trip_routes.pos", 'r') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['<\thidden_0\n', 'Kochi\thidden_2\n', 'Rishikesh\thidden_1\n', 'Hyderabad\thidden_4\n', 'Chennai\thidden_3\n']


In [3]:

with open("city_names.txt", 'r') as f:
    voc_l = f.read().split('\n')

print("A few items of the vocabulary list")
print(voc_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(voc_l[-50:])

A few items of the vocabulary list
['Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Bengaluru', 'Hyderabad', 'Ahmedabad', 'Jaipur', 'Agra', 'Varanasi', 'Amritsar', 'Kochi', 'Goa', 'Udaipur', 'Rishikesh', '<', '>', '']

A few items at the end of the vocabulary list
['Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Bengaluru', 'Hyderabad', 'Ahmedabad', 'Jaipur', 'Agra', 'Varanasi', 'Amritsar', 'Kochi', 'Goa', 'Udaipur', 'Rishikesh', '<', '>', '']


In [4]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 20:
        break

Vocabulary dictionary, key is the word, value is a unique integer
:0
<:1
>:2
Agra:3
Ahmedabad:4
Amritsar:5
Bengaluru:6
Chennai:7
Delhi:8
Goa:9
Hyderabad:10
Jaipur:11
Kochi:12
Kolkata:13
Mumbai:14
Rishikesh:15
Udaipur:16
Varanasi:17


In [5]:
vocab

{'': 0,
 '<': 1,
 '>': 2,
 'Agra': 3,
 'Ahmedabad': 4,
 'Amritsar': 5,
 'Bengaluru': 6,
 'Chennai': 7,
 'Delhi': 8,
 'Goa': 9,
 'Hyderabad': 10,
 'Jaipur': 11,
 'Kochi': 12,
 'Kolkata': 13,
 'Mumbai': 14,
 'Rishikesh': 15,
 'Udaipur': 16,
 'Varanasi': 17}

In [100]:
prep = ['Goa', 'Rishikesh']

In [101]:

def create_dictionaries(training_corpus, vocab):

    

    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    

    prev_tag = '--s--' 
    

    i = 0 
    

    for word_tag in training_corpus:
        

        i += 1
        
      
        if i % 50000 == 0:
            print(f"word count = {i}")
 
        word, tag = get_word_tag(word_tag, vocab)
        
     
        transition_counts[(prev_tag, tag)] += 1
        

        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

      
        prev_tag = tag
        

        
    return emission_counts, transition_counts, tag_counts

In [102]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [103]:
# get all the POS states
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 32
View these POS tags (states)
['hidden_0', 'hidden_1', 'hidden_10', 'hidden_11', 'hidden_12', 'hidden_13', 'hidden_14', 'hidden_15', 'hidden_16', 'hidden_17', 'hidden_18', 'hidden_19', 'hidden_2', 'hidden_20', 'hidden_21', 'hidden_22', 'hidden_23', 'hidden_24', 'hidden_25', 'hidden_26', 'hidden_27', 'hidden_28', 'hidden_29', 'hidden_3', 'hidden_30', 'hidden_31', 'hidden_4', 'hidden_5', 'hidden_6', 'hidden_7', 'hidden_8', 'hidden_9']


<a name='2'></a>
# Part 2: Hidden Markov Models for POS



<a name='2.1'></a>
## Part 2.1 Generating Matrices


In [104]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: create_transition_matrix
def create_transition_matrix(alpha, tag_counts, transition_counts):

    all_tags = sorted(tag_counts.keys())
    
    # Count the number of unique POS tags
    num_tags = len(all_tags)
    
    # Initialize the transition matrix 'A'
    A = np.zeros((num_tags,num_tags))
    
    # Get the unique transition tuples (previous POS, current POS)
    trans_keys = set(transition_counts.keys())
    
    ### START CODE HERE (Replace instances of 'None' with your code) ### 
    
    # Go through each row of the transition matrix A
    for i in range(num_tags):
        
        # Go through each column of the transition matrix A
        for j in range(num_tags):

            # Initialize the count of the (prev POS, current POS) to zero
            count = 0
        
            # Define the tuple (prev POS, current POS)
            # Get the tag at position i and tag at position j (from the all_tags list)
            key = (all_tags[i], all_tags[j])

            # Check if the (prev POS, current POS) tuple 
            # exists in the transition counts dictionary
            if key in transition_counts: #complete this line
                
                # Get count from the transition_counts dictionary 
                # for the (prev POS, current POS) tuple
                count = transition_counts[key]
                
            # Get the count of the previous tag (index position i) from tag_counts
            count_prev_tag = tag_counts[all_tags[i]]
            
            # Apply smoothing using count of the tuple, alpha, 
            # count of previous tag, alpha, and total number of tags
            A[i,j] = (count + alpha) / (count_prev_tag + alpha * num_tags)

    ### END CODE HERE ###
    
    return A

In [105]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.352840285
A at row 3, col 1: 0.0833
View a subset of transition matrix A
          hidden_8  hidden_9
hidden_8  0.000077  0.000077
hidden_9  0.380420  0.000048


### Create the 'B' emission probabilities matrix




In [106]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: create_emission_matrix

def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):

    
    # get the number of POS tag
    num_tags = len(tag_counts)
    
    # Get a list of all POS tags
    all_tags = sorted(tag_counts.keys())
    
 
    num_words = len(vocab)
    

    B = np.zeros((num_tags, num_words))
    

    emis_keys = set(list(emission_counts.keys()))

    for i in range(num_tags): # complete this line
        

        for j in range(num_words): # complete this line


            count = 0
       
            key =  (all_tags[i], vocab[j])


            if key in emission_counts: # complete this line
        
      
                count = emission_counts[key]
                

            count_tag = tag_counts[all_tags[i]]
                

            B[i,j] = (count + alpha) / (count_tag + alpha * num_words)


    return B

In [107]:
# creating your emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))



<a name='3'></a>
# Part 3: Viterbi Algorithm and Dynamic Programming



Represent infinity and negative infinity like this:

```CPP
float('inf')
float('-inf')
```

In [108]:

def initialize(states, tag_counts, A, B, corpus, vocab):
    
    num_tags = len(tag_counts)
    
    # Initialize best_probs matrix 

    best_probs = np.zeros((num_tags, len(corpus)))
    
    # Initialize best_paths matrix
 
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    # Define the start token
    s_idx = states.index("hidden_0")
 
    

    for i in range(num_tags): # complete this line
        
        
        if A[s_idx,i] == 0: # complete this line
            
            # Initialize best_probs at POS tag 'i', column 0, to negative infinity
            best_probs[i,0] = -float('inf')
        
   
        else:
            

            best_probs[i,0] = np.log(A[s_idx, i]) + np.log(B[i, vocab[corpus[0]]] )

    return best_probs, best_paths

In [109]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [110]:
best_paths

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [111]:
# Test the function
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -12.5746


IndexError: index 3 is out of bounds for axis 1 with size 2

<a name='3.2'></a>
## Part 3.2 Viterbi Forward



In [112]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_forward
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):

    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
    

    for i in range(1, len(test_corpus)): 


        for j in range(num_tags): # complete this line

            best_prob_i = -float('inf')
            

            best_path_i = None

       
            for k in range(num_tags): # complete this line
            

                prob = best_probs[k, i - 1] + math.log(A[k, j]) + math.log(B[j, vocab[test_corpus[i]]])

                # check if this path's probability is greater than
                # the best probability up to and before this point
                if prob > best_prob_i: # complete this line
                    
                    # Keep track of the best probability
                    best_prob_i = prob
                    
         
                    best_path_i = k


            best_probs[j,i] = best_prob_i
            
     
            best_paths[j,i] = best_path_i


    return best_probs, best_paths

In [113]:

best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

In [114]:
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -17.2580


IndexError: index 4 is out of bounds for axis 1 with size 2

##### Expected Output

```CPP
best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601
```

<a name='3.3'></a>
## Part 3.3 Viterbi backward



In [115]:
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_backward
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Get the number of words in the corpus
    # which is also the number of columns in best_probs, best_paths
    m = best_paths.shape[1] 
    
    # Initialize array z, same length as the corpus
    z = [None] * m
    
    # Get the number of unique POS tags
    num_tags = best_probs.shape[0]
    
    # Initialize the best probability for the last word
    best_prob_for_last_word = -float('inf')
    
    # Initialize pred array, same length as corpus
    pred = [None] * m

    for k in range(num_tags):


        if best_probs[k, m - 1] > best_prob_for_last_word:
            
            # Store the new best probability for the lsat word
            best_prob_for_last_word = best_probs[k, m - 1]
    
 
            z[m - 1] = k
            

    pred[m - 1] = states[z[m - 1]]

    for i in range(m - 1, -1, -1): # complete this line
        

        pos_tag_for_word_i = z[i]
        

        z[i - 1] = best_paths[pos_tag_for_word_i, i]
        
 
        pred[i - 1] = states[z[i - 1]]
        

    return pred

In [118]:
# Run and test your function
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['Goa'] 
 ['hidden_6'] 

The prediction for pred[0:8] is: 
 ['hidden_6', 'hidden_0'] 
 ['Goa', 'Rishikesh']


# read hidden encoders 

In [119]:
with open("hidden_encoders.txt", 'r') as f:
    hidden_states = f.readlines()
    

In [120]:
with open("routes.txt", 'r') as f:
    routes = f.readlines()




In [121]:
pred

['hidden_6', 'hidden_0']

In [122]:
f = []
hidden_stat = {}
for x in hidden_states:
    f.append(x.split('\t'))
    
    
    


In [123]:
hidden_stat = {}

for sublist in f:
    key = sublist[0]  # Extract the key
    value_str = sublist[1].strip()[1:-2]  # Extract the string of numbers and remove unnecessary characters
    value_list = [int(num) for num in value_str.split(', ')]  # Convert the string of numbers to a list of integers
    hidden_stat[key] = value_list  # Assign the value list to the dictionary key

print(hidden_stat)

{'hidden_0': [0, 1, 2, 3, 4, 5, 6, 10, 13, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 38, 39, 40, 41, 42, 43, 44, 55, 56, 57, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 82, 86, 87, 89, 91, 99, 10], 'hidden_1': [0, 1, 2, 3, 13, 21, 22, 23, 24, 26, 27, 28, 31, 32, 33, 41, 42, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 99, 10], 'hidden_2': [0, 1, 6, 10, 13, 17, 27, 28, 31, 32, 33, 38, 39, 40, 43, 44, 69, 70, 71, 72, 73, 74, 75, 82, 9], 'hidden_3': [0, 1, 6, 18, 31, 32, 33, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 86, 87, 8], 'hidden_4': [0, 1, 10, 13, 25, 26, 31, 32, 33, 66, 67, 68, 69, 70, 71, 72, 73, 89, 9], 'hidden_5': [1, 2, 3, 4, 5, 6, 13, 21, 22, 23, 24, 25, 26, 27, 28, 64, 65, 73, 7], 'hidden_6': [1, 2, 3, 4, 10, 13, 17, 21, 22, 25, 26, 27, 28, 38, 39, 40, 42, 43, 55, 56, 57, 73, 74, 9], 'hidden_7': [4, 10, 25, 26, 27, 28, 66, 67, 68, 87, 9], 'hidden_8': [4, 13, 26, 27, 28, 38, 39, 40, 41, 73, 74, 7], 'hidden_9': [4, 25, 26, 33, 38, 39, 40, 41, 42, 43, 44, 55, 56, 57, 73,

In [124]:
pred

['hidden_6', 'hidden_0']

find the intersection of routes

In [125]:
recommended_routes = set(hidden_stat[pred[0]]).intersection(hidden_stat[pred[1]])

In [126]:
for x in recommended_routes:
    print(routes[x])

['<', 'Rishikesh', 'Hyderabad', 'Chennai', 'Kochi', 'Kolkata', 'Goa', 'Rishikesh', '>']

['<', 'Kolkata', 'Goa', 'Rishikesh', 'Kolkata', '>']

['<', 'Goa', 'Rishikesh', 'Kolkata', 'Goa', '>']

['<', 'Kolkata', 'Goa', 'Delhi', 'Udaipur', 'Amritsar', 'Kolkata', '>']

['<', 'Delhi', 'Bengaluru', 'Mumbai', 'Kochi', 'Hyderabad', 'Goa', 'Delhi', '>']

['<', 'Kochi', 'Rishikesh', 'Amritsar', 'Kolkata', 'Goa', 'Hyderabad', 'Mumbai', 'Kochi', '>']

['<', 'Mumbai', 'Kochi', 'Goa', 'Mumbai', '>']

['<', 'Kolkata', 'Ahmedabad', 'Bengaluru', 'Agra', 'Goa', 'Rishikesh', 'Kolkata', '>']

['<', 'Bengaluru', 'Agra', 'Goa', 'Rishikesh', 'Kolkata', 'Bengaluru', '>']

['<', 'Kolkata', 'Goa', 'Udaipur', 'Hyderabad', 'Ahmedabad', 'Varanasi', 'Delhi', 'Kolkata', '>']

['<', 'Goa', 'Udaipur', 'Hyderabad', 'Ahmedabad', 'Varanasi', 'Delhi', 'Kolkata', 'Bengaluru', 'Agra', 'Amritsar', 'Rishikesh', 'Goa', '>']

['<', 'Varanasi', 'Delhi', 'Kolkata', 'Bengaluru', 'Agra', 'Amritsar', 'Rishikesh', 'Goa', 'Kochi', 'Va

<a name='4'></a>
# Part 4: Predicting on a data set

Compute the accuracy of your prediction by comparing it with the true `y` labels. 
- `pred` is a list of predicted POS tags corresponding to the words of the `test_corpus`. 

In [54]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

IndexError: list index out of range